# Cleaning data and visualizations

This notebook will serve as a quick primer to cleaning GIS data, mostly reprojecting, and how to overlay layers to make static maps.

Most geographical analysis can be done using Geopandas you can read more about the library and see its documentation [here](https://geopandas.org/en/stable/docs.html)

## 1. Imports

First, let's import the necessary libraries to work with geographical data and make some visualizations

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import rasterio
import numpy as np
import contextily as cx

## 2. Loading GIS Data

Let's start by loading some sample GIS data. We'll load one of the already cleaned datasets, bus stops in LA.

When reading in GIS data, geopandas will automatically detect what type of file it is and if it has geometry attributes

A 'geometry' column is what will seperate a GeoDataFrame from a typical pandas dataframe. 

A GeoDataFrame as all the same functionality a normal pandas dataframe has with added methods for working with GIS data

In [ ]:
# Load in a GeoJSON

bus_stops = gpd.read_file("/Users/jaycampanell/Downloads/ShadeLA Layers/bus_stops.geojson")

bus_stops.head()

Regardless of the format of the data it is a similar process for shapefiles for example

For shapefiles, they typically come from a zip file and to load them in just unzip it and
use the same gpd.read_file() function to read it in. 

**IMPORTANT:**
For shapefiles the directory which the shapefile comes in must also contain 4 other files:
1. .cpg
2. .dbf
3. .prj
4. .shx

Otherwise geopandas will not be able to read in the file, but once you read it in, everything will function the same. 

In [ ]:

bus_stops_shp = gpd.read_file("/Users/jaycampanell/Documents/shadela/461/data/bus_stops/bus_stops.shp")

bus_stops_shp.head()

## 3. Exploring Data Properties

Before cleaning, it's important to understand the properties of your data.

All GIS data comes in a certain map projection, or coordinate reference system (CRS). Typically if you are downloading your data from online it will be in the standard Web Mercator Projection (EPSG:4326), but sometimes it won't and if the data is not in the same projection, it makes it much more difficult to visualize it. 

In [ ]:
# Let's check that our bus stops are in the right projection

print(f"Bus Stops CRS: {bus_stops.crs}")

# But what if the data isn't in the right projection?
# You can use the built in "to_crs()" function

# Let's look at the excess rate of ER visits

excess_er = gpd.read_file("/Users/jaycampanell/Downloads/ShadeLA Layers/la_excess_er.geojson")


print(f"Excess ER Visits CRS: {excess_er.crs}")

# Now that we see it isn't in the right projection we can convert it to the standard Web Mercator Projection

excess_er_reprojected = excess_er.to_crs("EPSG:4326")

print(f"Corrected Excess ER Visits CRS: {excess_er_reprojected.crs}")

## 4. Creating Static Maps

Now let's actually make some maps. 

To make static maps you still use the standard logic of matplotlib.

In [ ]:
# Let's make a map of bus stops and excess er visits


fig, ax = plt.subplots(figsize = (12, 10))


# To make a choropleth map and show attribute data on the polygon data use the 'column' argument
excess_er_reprojected.plot(ax = ax, label = "Excess ER Visits", column = 'excs_ER',cmap='YlOrRd', legend = True)
bus_stops.plot(ax = ax, label = "Bus Stops", color = 'purple', markersize = 1)

ax.set_title('Choropleth Map - Rate (per 10,000) of Excess ER Visits on Hot Days', fontsize=16)
ax.set_xlabel('Easting (meters)', fontsize=12)
ax.set_ylabel('Northing (meters)', fontsize=12)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.legend()
plt.show()

# 5. Sampling polygon values at specific points

If you wanted to actually get the value of Excess ER Visits at each of those bus stops from the polygon layer you can use a spatial join.

Spatial joins are similar to traditional relational database joins but instead of joining on some primary and foreign key, you can use the geographies of the different data points.

There are multiple predicates, or ways you can join, and you can read the documentation [here](https://geopandas.org/en/stable/docs/reference/api/geopandas.sjoin.html)

Common predicates include:
Intersection - where there is overlap for both geometries (the default if nothing else is specified)
Contains - Typically used when you have a polygon and want to find all the points within that polygon
Within - Vice versa of contains, when wanting to find the points within a polygon

One last thing to note, order matters in the join so if you put your polygon geodataframe first as an argument, then the point, the logic would be polygon contains point so you'd have to use either "Intersection" or "Contains" as the predicate. The logic polygon within point, wouldn't be valid and would yield an empty df or one with many missing values.


In [ ]:
# This will do a join on all the bus stops with the polygon layers based on the intersection (the predicate)

bus_stops_excess_er = gpd.sjoin(bus_stops, excess_er_reprojected, how = 'left') # make sure you join dataframes that are in the same projection

# Now we can visualize the excess number of er visits by bus stops, although since the aggregation 
# is a the zip code level, there won't be much variation

fig, ax = plt.subplots(figsize = (12, 10))

bus_stops_excess_er.plot(ax = ax, label = "Bus Stops", column = 'excs_ER',cmap='YlOrRd', markersize = 1, legend = True)

# For this map let's turn off the axis and add LA as the background map
cx.add_basemap(ax, source=cx.providers.CartoDB.PositronNoLabels, crs = bus_stops_excess_er.crs)
# if you are interested in more background/basemaps you can see the full list with the command cx.providers

ax.set_axis_off()
ax.set_title('Rate (per 10,000) of Excess ER Visits on Hot Days at Bus Stop Level', fontsize=16)


ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.legend()
plt.show()


## 6. Working with Raster Data

Raster data is data that is represented by a grid of pixels, instead of being at a specific location (point, polygon, line) like much of the data we work with. This means we need a different set of tools to understand and visualize this.

Raster data contains different 'bands' which actually contains the numerical pixel value. In python, this manifests as numpy arrays.

The raster dataset in this project is from NASA JPL's ECOSTRESS project which utilizes a Thermal Infrared Radiometer to measure heat on Earth. We are interested in heat during summer. This image measures Land Surface Temperature (LST) during summer in LA. It is the average temperature from 2018-2023 from the hours of 1-4pm. La

In [ ]:
# Rasterio is a libray used to process raster data. 

# First open the tif file
with rasterio.open("./data/ECOSTRESS_LST.tif") as src:

    # read in the first band
    data = src.read(1)
    
    # create a mask where there is NaN data
    data_masked = np.ma.masked_invalid(data)
    
    plt.figure(figsize=(12, 10))
    
    # Create colormap and set the color for masked values
    cmap = plt.cm.Spectral_r

    # Set NaN/masked areas to black
    cmap.set_bad(color='black')  
    
    im = plt.imshow(data_masked, cmap=cmap, vmin=0, vmax=61.38, interpolation='none')
    plt.colorbar(im, label='Values')
    plt.title('ECOSTRESS Street LST Data')
    plt.tight_layout()
    plt.show()



Since this raster data is essentially a numpy array with indicies of coordinate points, we can sample it for those coordinate points. 

So let's try to find the LST at all of our bus stops.

In [ ]:
# Open the geotif with rasterio
with rasterio.open('./data/ECOSTRESS_LST.tif') as src:
    # Reproject if CRS doesn't match
    if bus_stops.crs != src.crs:
        bus_stops = bus_stops.to_crs(src.crs)
    
    # Extract coordinates from Point geometries
    coords = [(geom.x, geom.y) for geom in bus_stops.geometry]
    
    # Sample the raster
    bus_stops['LST'] = [v[0] for v in src.sample(coords)]

print(bus_stops[['geometry', 'LST']])